In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
df = pd.read_csv('Dataset.csv')
G = nx.from_pandas_edgelist(df, 'user_id', 'item_id', edge_attr='rating')
adj_matrix = nx.adjacency_matrix(G)


In [ ]:
print(adj_matrix.todense())

[[0 1 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
user_ids = df['user_id'].unique()
movie_ids = df['item_id'].unique()
num_users = len(user_ids)
num_movies = len(movie_ids)
user_id_map = {id: i for i, id in enumerate(user_ids)}
movie_id_map = {id: i for i, id in enumerate(movie_ids)}

In [ ]:
import torch.nn.functional as F
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.gc = nn.Linear(input_dim, hidden_dim)
        self.activation = nn.ReLU()
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, adj):
        x = self.gc(torch.matmul(adj, x))
        x = self.activation(x)
        x = self.fc(torch.matmul(adj, x))
        x = F.softmax(x, dim=output_dim) 
        return x

input_dim = num_users + num_movies
hidden_dim = 64
output_dim = 5

model = GCN(input_dim, hidden_dim, output_dim)


In [ ]:
def train(model, adj, features, labels):
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

num_epochs = 100


In [ ]:
import torch.optim as optim

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Convert the adjacency matrix to a dense tensor
adj_matrix = adj_matrix.todense()

# Convert the data to PyTorch tensors
inputs = torch.tensor(adj_matrix, dtype=torch.float32)
labels = torch.tensor(df['rating'].values, dtype=torch.long)

# Training loop
epochs = 100
for epoch in range(epochs):
    # Zero the gradients
    optimizer.zero_grad()

    # Forward pass
    outputs = model(inputs, inputs)

    # Compute the loss
    loss = criterion(outputs, labels)

    # Backward pass
    loss.backward()

    # Update the weights
    optimizer.step()

    # Print the loss for every few epochs
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

# After training, you can use the model for predictions


AttributeError: ignored

## To understand

In [ ]:
import pandas as pd
import networkx as nx
import numpy as np

df = pd.DataFrame({
    'user_id': [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
    'item_id': [50, 172, 133, 61, 189, 33, 160, 20, 202, 171],
    'rating': [5, 5, 1, 4, 3, 4, 4, 4, 5, 5],
    'timestamp': [881250949, 881250949, 881250949, 878542420, 888732928, 878542699, 875072547, 887431883, 875072442, 889751711]
})

G = nx.from_pandas_edgelist(df, 'user_id', 'item_id', edge_attr='rating')
adj_matrix = nx.adjacency_matrix(G)

print(adj_matrix.todense())


[[0 1 1 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 1 1]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0]]


In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Charger le dataset depuis le fichier CSV
df = pd.read_csv('Dataset.csv')

# Créer le graphe à partir des données
G = nx.from_pandas_edgelist(df, 'user_id', 'item_id', edge_attr='rating')

# Calculer la matrice d'adjacence
adj_matrix = nx.adjacency_matrix(G)

# Obtenir les IDs uniques pour les utilisateurs et les films
user_ids = df['user_id'].unique()
movie_ids = df['item_id'].unique()

# Nombre total d'utilisateurs et de films
num_users = len(user_ids)
num_movies = len(movie_ids)

# Créer une table de hachage pour mapper les IDs aux indices
user_id_map = {id: i for i, id in enumerate(user_ids)}
movie_id_map = {id: i for i, id in enumerate(movie_ids)}

# Définir les dimensions d'entrée, cachée et de sortie pour le modèle GCN
input_dim = num_users + num_movies
hidden_dim = 64
output_dim = 5

# Définir le modèle GCN
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.gc = nn.Linear(input_dim, hidden_dim)
        self.activation = nn.ReLU()
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, adj):
        x = self.gc(torch.matmul(x, adj))
        x = self.activation(x)
        x = self.fc(x)
        x = F.softmax(x, dim=1)
        return x

# Instancier le modèle GCN
model = GCN(input_dim, hidden_dim, output_dim)

# Définir la fonction d'entraînement
def train(model, adj, features, labels):
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()
    return loss.item()


In [ ]:
# Définir le nombre d'époques et les paramètres d'optimisation
num_epochs = 100
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Convertir la matrice d'adjacence en tenseur PyTorch
adj_matrix = adj_matrix.todense()
adj_tensor = torch.tensor(adj_matrix, dtype=torch.float32)

# Convertir les labels en tenseur PyTorch
labels = torch.tensor(df['rating'].values, dtype=torch.long)

# Entraînement du modèle
for epoch in range(num_epochs):
    # Réinitialiser les gradients
    optimizer.zero_grad()

    # Effectuer une passe avant (forward pass)
    features = torch.eye(input_dim)

# Utiliser le modèle pour les prédictions
    model.eval()
    with torch.no_grad():
        outputs = model(features, adj_tensor)

# Obtenir les prédictions sous forme de labels
    predicted_labels = torch.argmax(outputs, dim=1)

# Afficher les prédictions
    print(predicted_labels)

    # Calculer la perte (loss)
    loss = criterion(outputs, labels)

    # Effectuer une passe arrière (backward pass) et mettre à jour les poids
    loss.backward()
    optimizer.step()

    # Afficher la perte pour chaque époque
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Utilisation du modèle pour les prédictions
outputs = model(adj_tensor, adj_tensor)


RuntimeError: ignored